In [3]:
import yfinance as yf
import pandas as pd

In [31]:
sp500 = yf.download('^GSPC', start='2015-01-01', end='2024-12-31')
sp500 = sp500[['Close']]
sp500['Return'] = sp500['Close'].pct_change()

# Label: 1 = up, -1 = down, 0 = flat
sp500['Label'] = sp500['Return'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
sp500 = sp500.dropna()
print(sp500.head())

C:\Users\Kiki\AppData\Local\Temp\ipykernel_20744\3289658041.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500 = yf.download('^GSPC', start='2015-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed

Price             Close    Return Label
Ticker            ^GSPC                
Date                                   
2015-01-05  2020.579956 -0.018278    -1
2015-01-06  2002.609985 -0.008893    -1
2015-01-07  2025.900024  0.011630     1
2015-01-08  2062.139893  0.017888     1
2015-01-09  2044.810059 -0.008404    -1


In [14]:

import pandas as pd
import requests
from io import BytesIO
import zipfile
from datetime import datetime, timedelta

# Define the relevant columns and keywords
GDELT_COLUMNS = [
    "GLOBALEVENTID", "SQLDATE", "MonthYear", "Year", "Actor1Name", "Actor2Name",
    "EventCode", "EventBaseCode", "EventRootCode", "QuadClass", "GoldsteinScale",
    "NumMentions", "AvgTone", "SOURCEURL"
]

# Keywords to keep (feel free to expand)
RELEVANT_KEYWORDS = ['financial', 'economy', 'war', 'conflict', 'trade', 'sanction',
                     'market', 'inflation', 'bank', 'military', 'geopolitical']

# Prepare date range
start_date = pd.to_datetime("2015-01-01")
end_date = pd.to_datetime("2015-01-11")
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Function to get GDELT data for a specific day
def get_gdelt_data(date):
    ymd = date.strftime("%Y%m%d")
    url = f"http://data.gdeltproject.org/events/{ymd}.export.CSV.zip"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            print(f"No data for {ymd}")
            return pd.DataFrame()

        with zipfile.ZipFile(BytesIO(response.content)) as z:
            with z.open(z.namelist()[0]) as f:
                df = pd.read_csv(f, header=None, sep='\t', encoding='latin1', low_memory=False)
                df.columns = [f'col{i}' for i in range(df.shape[1])]  # GDELT has ~58 columns
                return df
    except Exception as e:
        print(f"Error fetching {ymd}: {e}")
        return pd.DataFrame()

# Function to filter for relevant events
def process_gdelt_day(df):
    df = df.copy()
    # Keep only relevant columns if they exist
    df = df.rename(columns={
        'col0': 'GLOBALEVENTID',
        'col1': 'SQLDATE',
        'col2': 'MonthYear',
        'col3': 'Year',
        'col7': 'Actor1Name',
        'col16': 'Actor2Name',
        'col26': 'EventCode',
        'col27': 'EventBaseCode',
        'col28': 'EventRootCode',
        'col29': 'QuadClass',
        'col30': 'GoldsteinScale',
        'col31': 'NumMentions',
        'col34': 'AvgTone',
        'col57': 'SOURCEURL'
    })

    # Only keep defined columns
    df = df[[col for col in GDELT_COLUMNS if col in df.columns]]

    # Keyword filtering on Actor1, Actor2, and SourceURL
    filter_mask = df['Actor1Name'].fillna('').str.lower().str.contains('|'.join(RELEVANT_KEYWORDS)) | \
                  df['Actor2Name'].fillna('').str.lower().str.contains('|'.join(RELEVANT_KEYWORDS)) | \
                  df['SOURCEURL'].fillna('').str.lower().str.contains('|'.join(RELEVANT_KEYWORDS))

    return df[filter_mask]

all_days = []
for i, date in enumerate(date_range):
    df = get_gdelt_data(date)
    if not df.empty:
        filtered = process_gdelt_day(df)
        if not filtered.empty:
            all_days.append(filtered)
    if i % 100 == 0:
        print(f"Processed {i} days...")

if all_days:
    gdelt_final = pd.concat(all_days, ignore_index=True)
    print(f"Final GDELT dataset shape: {gdelt_final.shape}")
    gdelt_final.to_csv("gdelt_filtered_2015_2024.csv", index=False)
else:
    print("No data collected.")


Processed 0 days...
✅ Final GDELT dataset shape: (81854, 14)


In [15]:
df.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57
0,331284899,20050113,200501,2005,2005.0356,GOV,ADMINISTRATION,NaN,NaN,NaN,...,1503239,5,"Heilongjiang, Heilongjiang, China",CH,CH08,48.0000,128.0000,-1908709,20150111,http://www.vnews.com/news/nation/world/1506762...
1,331284900,20050113,200501,2005,2005.0356,THA,THAILAND,THA,NaN,NaN,...,-3250083,4,"Bangkok, Krung Thep Mahanakhon, Thailand",TH,TH40,13.7500,100.5170,-3250083,20150111,http://www.nationmultimedia.com/national/Priva...
2,331284901,20050113,200501,2005,2005.0356,USA,WASHINGTON,USA,NaN,NaN,...,-1898541,4,"Beijing, Beijing, China",CH,CH22,39.9289,116.3880,-1898541,20150111,http://www.abqjournal.com/524406/news/a-commod...
3,331284902,20050113,200501,2005,2005.0356,USA,WASHINGTON,USA,NaN,NaN,...,-661412,4,"Perus, SÃ£Paulo, Brazil",BR,BR27,-23.4058,-46.7463,-661412,20150111,http://www.abqjournal.com/524406/news/a-commod...
4,331284903,20140111,201401,2014,2014.0301,NaN,NaN,NaN,NaN,NaN,...,-3378435,4,"Kabul, Kabol, Afghanistan",AF,AF13,34.5167,69.1833,-3378435,20150111,http://www.tolonews.com/en/afghanistan/17797-m...


In [29]:
gdelt_final.head()

,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,Actor1Name,Actor2Name,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,AvgTone,SOURCEURL,date
0,0,330163377,20050103,200501,2005,CAN,UNITED STATES,61,61,6,2,6.4,2,2.785515,http://ottawacitizen.com/news/national/daniell...,2005-01-03
1,1,330163378,20050103,200501,2005,CAN,UNITED STATES,61,61,6,2,6.4,8,2.785515,http://ottawacitizen.com/news/national/daniell...,2005-01-03
2,2,330163426,20140101,201401,2014,NaN,WEST BENGAL,36,36,3,1,4.0,5,0.000000,http://www.business-standard.com/article/marke...,2014-01-01
3,3,330163438,20140101,201401,2014,NaN,INFOSYS,43,43,4,1,2.8,10,2.127660,http://economictimes.indiatimes.com/markets/st...,2014-01-01
4,4,330163462,20140101,201401,2014,NaN,UNITED STATES,190,190,19,4,-10.0,4,2.948718,http://www.delawareonline.com/story/news/local...,2014-01-01


In [17]:
gdelt_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81854 entries, 0 to 81853
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   GLOBALEVENTID   81854 non-null  int64  
 1   SQLDATE         81854 non-null  int64  
 2   MonthYear       81854 non-null  int64  
 3   Year            81854 non-null  int64  
 4   Actor1Name      47089 non-null  object 
 5   Actor2Name      62667 non-null  object 
 6   EventCode       81854 non-null  int64  
 7   EventBaseCode   81854 non-null  int64  
 8   EventRootCode   81854 non-null  int64  
 9   QuadClass       81854 non-null  int64  
 10  GoldsteinScale  81854 non-null  float64
 11  NumMentions     81854 non-null  int64  
 12  AvgTone         81854 non-null  float64
 13  SOURCEURL       81854 non-null  object 
dtypes: float64(2), int64(9), object(3)
memory usage: 8.7+ MB


In [27]:
gdelt_final['date'] = pd.to_datetime(gdelt_final['SQLDATE'], format='%Y%m%d')


In [44]:
sp500 = sp500.reset_index()


In [47]:
sp500.head()

,date,close,return,label
0,2015-01-05,2020.579956,-0.018278,-1
1,2015-01-06,2002.609985,-0.008893,-1
2,2015-01-07,2025.900024,0.011630,1
3,2015-01-08,2062.139893,0.017888,1
4,2015-01-09,2044.810059,-0.008404,-1


In [46]:
sp500.columns = ['date', 'close', 'return', 'label']


In [48]:
merged = pd.merge(gdelt_final, sp500, on='date')


In [50]:
merged.head()

,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,Actor1Name,Actor2Name,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,AvgTone,SOURCEURL,date,close,return,label
0,24960,330534530,20150105,201501,2015,NaN,AFGHANISTAN,37,37,3,1,5.0,2,3.716216,http://news.tj/en/news/draft-transit-trade-dea...,2015-01-05,2020.579956,-0.018278,-1
1,24961,330534531,20150105,201501,2015,NaN,AFGHANISTAN,37,37,3,1,5.0,8,3.716216,http://news.tj/en/news/draft-transit-trade-dea...,2015-01-05,2020.579956,-0.018278,-1
2,24962,330534534,20150105,201501,2015,NaN,AFGHANISTAN,42,42,4,1,1.9,12,1.057065,http://www.navytimes.com/story/military/tech/2...,2015-01-05,2020.579956,-0.018278,-1
3,24963,330534545,20150105,201501,2015,NaN,AFGHANISTAN,51,51,5,1,3.4,11,3.162798,http://www.columbian.com/news/2015/jan/05/supp...,2015-01-05,2020.579956,-0.018278,-1
4,24964,330534554,20150105,201501,2015,NaN,AFGHANISTAN,112,112,11,3,-2.0,14,1.984410,https://www.indianagazette.com/news/reg-nation...,2015-01-05,2020.579956,-0.018278,-1
